# Azure Storage: Append Blob Latency 

### Default Credential

In [ ]:
from azure.cli.core import get_default_cli
get_default_cli().invoke(["account", "show"])

In [ ]:
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

In [ ]:
from azure.storage.blob import BlobServiceClient
storage_url = "https://XXXXXXXXXXXXXXXXXXXXXXXXXX.blob.core.windows.net/"
blob_service_client = BlobServiceClient(account_url=storage_url, credential=credential)
containers = blob_service_client.list_containers()
container_name = "XXXXXXXXXXXXXXXXXXXXXX"
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
results = []
for blob in container_client.list_blobs():
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)
    blob_properties = blob_client.get_blob_properties()
    last_modified_date = blob_properties.last_modified
    created_date = blob_properties.creation_time
    date_difference = (last_modified_date - created_date).total_seconds() / 60
    results.append([blob.name, created_date, last_modified_date, date_difference])

In [ ]:
import pandas as pd
df = pd.DataFrame(results, columns=["filename","created_date","modified_date","difference_min"])
df["difference_min"].describe()

### SAS Token 

In [ ]:
# note: give SAS token READ and LIST permissions
from azure.storage.blob import BlobServiceClient 
storage_url = "https://XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX.blob.core.windows.net/"
sas_token = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
blob_service_client = BlobServiceClient(account_url=storage_url, credential=sas_token)
container_client = blob_service_client.get_container_client("XXXXXXXXXXXXXXXXXXXXXXXX")

In [ ]:
results = [] 
for each_blob in container_client.list_blobs():
    last_modified_date = each_blob.last_modified
    created_date = each_blob.creation_time
    date_difference = (last_modified_date - created_date).total_seconds() / 60
    results.append([each_blob.name, created_date, last_modified_date, date_difference])

In [ ]:
import pandas as pd
df = pd.DataFrame(results, columns=["filename","created_date","modified_date","difference_min"])
df["difference_min"].describe()

In [ ]:
for each in [0.95, 0.96, 0.965, 0.97, 0.975, 0.98, 0.985, 0.99, 0.995, 0.999, 1.0]: 
    each_quantile = df["difference_min"].quantile(each)
    print(f"Quantile: {each}, Latency: {each_quantile} min")